This notebook begins the steps for implementing the data into a RAG system. We give the LLM the user-inputed lyrics, and we then find the embedding, use FAISS to compare and grab k similar lyrics from our data, and the LLM predicts popularity and explain why given the popularity and audio features of the similar songs. 

There is sitll work to be done:

- we use the LLM for prediciton and explinations. using a prediction model instead can lead to better reproducibility (consistant scores)
- we currently use dummy lyric embedding and FAISS similarity functions
- better prompting techniques
- deep eval to evaluate the perforamnce of RAG

### Downloading the model first

In [1]:
import os
import shutil
from pathlib import Path

# --- FIX 1: Silence the Progress Bar (Aggressive Mode) ---
# This disables ALL progress bars, which prevents the 'IProgress' crash
os.environ["TQDM_DISABLE"] = "1"

# --- FIX 2: Delete the Corrupted Cache ---
# We need to remove the broken download from the previous attempt
cache_dir = Path(os.path.expanduser("~/.cache/huggingface/hub"))
model_dir = cache_dir / "models--sentence-transformers--all-MiniLM-L6-v2"

if model_dir.exists():
    print(f"Removing corrupted cache at: {model_dir}")
    shutil.rmtree(model_dir)
else:
    print("Cache was clean (no corrupted files found).")

# --- NOW LOAD THE MODEL ---
print("Downloading model... (This will be silent, wait ~20 seconds)")
import faiss
from sentence_transformers import SentenceTransformer

# This should now run cleanly
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Success! Model loaded.")

Cache was clean (no corrupted files found).


/ext3/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Success! Model loaded.


### Reading in data and making df

In [4]:
import numpy as np
import pandas as pd
import pandas as pd
from typing import List, Dict, Any


pickle1 = pd.read_pickle("data/lyric_embeddings/librosa_shard_0_clean_ml_with_emb.pkl")
pickle2 = pd.read_pickle("data/lyric_embeddings/librosa_shard_1_clean_ml_with_emb.pkl")
pickle3 = pd.read_pickle("data/lyric_embeddings/librosa_shard_2_clean_ml_with_emb.pkl")
pickle4 = pd.read_pickle("data/lyric_embeddings/librosa_shard_3_clean_ml_with_emb.pkl")
pickle5 = pd.read_pickle("data/lyric_embeddings/librosa_shard_4_clean_ml_with_emb.pkl")

df = pd.concat([pickle1, pickle2, pickle3, pickle4, pickle5])

### Building FAISS Index and using top 50 rows since we don't have the actual lyric embeddings yet

In [5]:
import faiss
import numpy as np

print("Building temporary index from top 50 songs...")

# 1. Grab the top 50 rows from the dataframe
# We use .copy() to ensure we don't affect the original dataframe
df_subset = df.head(50).copy()

# 2. Combine title + artist + lyrics for the vector
# We handle non-string data just in case by forcing .astype(str)
subset_text = (
    df_subset['title'].astype(str) + " " +
    df_subset['artist'].astype(str) + " " +
    df_subset['lyrics'].astype(str).str.slice(0, 500)
).tolist()

# 3. Generate vectors using the model you just loaded
subset_vectors = model.encode(subset_text)

# 4. Build FAISS Index
dimension = subset_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(subset_vectors.astype('float32'))

print(f"Success! Created index with {index.ntotal} songs.")

Building temporary index from top 50 songs...
Success! Created index with 50 songs.


### REAL embedding functions

In [6]:
import faiss

emb_list = [np.asarray(x, dtype="float32") for x in df["lyrics_embedding"].values]
emb_matrix = np.stack(emb_list, axis=0)
dimension = emb_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)

index.add(emb_matrix.astype("float32"))

print("FAISS index built with", index.ntotal, "vectors.")

def retrieve_similar_songs(query_embedding: np.ndarray, k: int = 5) -> List[Dict[str, Any]]:
    query_vector = np.array([query_embedding]).astype('float32')
    D, I = index.search(query_vector, k)

    neighbors = []
    for idx, dist in zip(I[0], D[0]):
        if idx != -1:
            neighbors.append({
                "index": int(idx),
                "similarity": float(dist)
            })

    return neighbors


In [7]:
audio_feature_cols = list(df.columns[df.columns.get_loc("duration")+1:])
df.head()

,song_id,title,artist,query_title,query_artist,track_genre,popularity,lyrics,preview_url,track_id,...,spectral_contrast_6,spectral_contrast_7,tonnetz_1,tonnetz_2,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6,lyrics_clean,lyrics_embedding
0,4845,State of Mind,Scooter,state of mind,scooter,happy,24.0,The world seems not the same...\n\nIntroducing...,https://audio-ssl.itunes.apple.com/itunes-asse...,1692327616,...,18.328021,39.053367,0.197966,-0.116721,0.142559,-0.069539,-0.044986,-0.047523,the world seems not the same introducing twist...,"[0.07519828, -0.0233649, -0.06524662, -0.07315..."
1,462,Reptilia,The Strokes,reptilia,the strokes,alt-rock,75.0,[Verse 1]\nHe seemed impressed by the way you ...,https://audio-ssl.itunes.apple.com/itunes-asse...,302987569,...,17.382681,39.012014,0.078138,-0.077754,0.063345,0.036541,-0.011976,-0.014041,he seemed impressed by the way you came in tel...,"[-0.08670999, -0.025700577, -0.08122497, -0.02..."
2,16017,None Of My Business,Cher Lloyd,none of my business,cher lloyd,electro,64.0,"[Chorus]\nDamn, I heard that you and her been ...",https://audio-ssl.itunes.apple.com/itunes-asse...,1438630505,...,18.248683,39.966514,0.013912,0.172900,-0.092766,-0.056323,-0.004173,-0.014388,damn i heard that you and her been having prob...,"[0.017929412, 0.0015679213, 0.00086991367, -0...."
3,9478,Trouble Sleeping,The Perishers,trouble sleeping,the perishers,acoustic,48.0,I'm having trouble sleeping\nYou're jumping in...,https://audio-ssl.itunes.apple.com/itunes-asse...,89335271,...,16.969837,28.947224,-0.118755,0.195544,0.025169,-0.130705,0.024176,0.005865,i'm having trouble sleeping you're jumping in ...,"[0.012034113, -0.0008498362, -0.040335782, 0.0..."
4,2822,Shot in the Dark,Ozzy Osbourne,shot in the dark,ozzy osbourne,hard-rock,65.0,[Verse 1]\nOut on the streets I'm stalking the...,https://audio-ssl.itunes.apple.com/itunes-asse...,158711416,...,17.184653,35.540522,-0.113671,0.023209,-0.029743,-0.051142,0.003486,-0.011837,out on the streets i'm stalking the night i ca...,"[-0.054401744, 0.021241566, -0.05175488, -0.01..."


In [8]:
def get_top_k_neighbors(df, query_embedding, k=5):
    raw_neighbors = retrieve_similar_songs(query_embedding, k=k)
    neighbors = []

    for n in raw_neighbors:
        idx = n["index"]
        row = df.iloc[idx]

        audio_features = {}

        for col in audio_feature_cols:
            val = row[col]

            # keep if scalar
            if np.isscalar(val):
                audio_features[col] = float(val)
            
            # flatten if array
            elif isinstance(val, np.ndarray):
                val = val.flatten()
                for j, v in enumerate(val):
                    audio_features[f"{col}_{j}"] = float(v)
            
            # flatten if list
            elif isinstance(val, list):
                for j, v in enumerate(val):
                    audio_features[f"{col}_{j}"] = float(v)

            else:
                try:
                    audio_features[col] = float(val)
                except Exception:
                    audio_features[col] = None

        neighbor_data = {
            "song_id": row["song_id"],
            "title": row["title"],
            "artist": row["artist"],
            "similarity": n.get("similarity", None),
            "popularity": float(row["popularity"]),
            "lyrics_snippet": row["lyrics"][:400].replace("\n", " ") + "...",
            "audio_features": audio_features
        }
        
        neighbors.append(neighbor_data)

    return neighbors


In [9]:
# continue working on promping
def build_rag_prompt_for_lyric_popularity(user_lyric: str,neighbors: List[Dict[str, Any]]) -> str:

    lines = []
    lines.append("You are an expert in music analytics, audio features, and lyric interpretation.")
    lines.append("You are given a NEW lyric and several similar songs from a dataset.")
    lines.append("")
    lines.append("Each similar song includes:")
    lines.append(" - song_id, title, artist")
    lines.append(" - lyric snippet")
    lines.append(" - popularity score (0-100)")
    lines.append(" - detailed audio features extracted from 30-second clips")
    lines.append("")
    lines.append("Your tasks are:")
    lines.append("  1. Predict a popularity score (0-100) for the NEW lyric.")
    lines.append("  2. Explain your reasoning using comparisons to the similar songs.")
    lines.append("")
    lines.append("Keep in mind the audio features of the similar songs, and explain what they mean in context to everyday people.")
    lines.append("")
    lines.append("Return your answer as VALID JSON with this exact format:")
    lines.append("{")
    lines.append('  "predicted_popularity": <number>,')
    lines.append('  "explanation": "<multi-paragraph explanation grounded in the provided songs>"')
    lines.append("}")
    lines.append("")
    lines.append("IMPORTANT:")
    lines.append("Return ONLY raw JSON.")
    lines.append("Do NOT include any code fences such as ``` json")
    lines.append("Do NOT include any explanation text outside the JSON.")
    lines.append("Do NOT add commentary before or after the JSON.")
    lines.append("Return JSON ONLY.")
    lines.append("")
    lines.append("------------------------------------------------------------")
    lines.append("NEW LYRIC:")
    lines.append(user_lyric.strip())
    lines.append("------------------------------------------------------------")
    lines.append("")
    lines.append("SIMILAR SONGS FROM THE DATASET (use these as evidence):")

    for i, nb in enumerate(neighbors, start=1):
        lines.append(f"\nNeighbor #{i}:")
        lines.append(f"  song_id: {nb['song_id']}")
        lines.append(f"  title: {nb['title']}")
        lines.append(f"  artist: {nb['artist']}")
        if nb["similarity"] is not None:
            lines.append(f"  similarity: {nb['similarity']:.4f}")
        lines.append(f"  popularity: {nb['popularity']:.2f}")
        lines.append(f"  lyrics_snippet: {nb['lyrics_snippet']}")
        lines.append("  audio_features:")

        for feat_name, feat_val in nb["audio_features"].items():
            lines.append(f"    {feat_name}: {feat_val:.4f}")

    lines.append("")
    lines.append(
        "Using ONLY the information above, estimate the popularity of the new lyric "
        "and explain your reasoning in terms of lyric similarity, artist/genre patterns, "
        "and audio features (energy, brightness, tempo, chroma, MFCCs, contrasts, tonnetz, etc.)."
        "Make sure to contextulize what the audio features mean for the average person."
    )

    return "\n".join(lines)


In [10]:
import os
import re
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

def call_llm_for_popularity_and_explanation(prompt: str) -> dict:

    response = client.responses.create(
        model="gpt-4o",
        input=prompt,
        temperature=0.2,
        max_output_tokens=900
    )

    raw_text = response.output[0].content[0].text.strip()

    # Remove any ```json ...``` or ```
    raw_text = raw_text.replace("```json", "")
    raw_text = raw_text.replace("```", "")
    raw_text = raw_text.strip()

    # first try direct json parse
    try:
        return json.loads(raw_text)
    except:
        pass

    # else, find outside json block using regex
    json_matches = re.findall(r"\{(?:[^{}]|(?:\{[^{}]*\}))*\}", raw_text, flags=re.DOTALL)

    if json_matches:
        for match in json_matches:
            try:
                return json.loads(match)
            except:
                continue

    # try to repair json with trailing commas
    repaired = re.sub(r",\s*([}\]])", r"\1", raw_text)

    try:
        return json.loads(repaired)
    except:
        pass

    # else, say failed
    print("Could not parse JSON from LLM output. Returning raw text.")
    return {
        "predicted_popularity": None,
        "explanation": raw_text
    }





ModuleNotFoundError: No module named 'openai'

In [ ]:
def rag_lyric_popularity_system(df: pd.DataFrame, user_lyric: str, k_neighbors: int = 5) -> Dict[str, Any]:
    # 1) embed
    query_embedding = embed_lyrics(user_lyric)

    # 2) find songs with similar sounding lyrics
    neighbors = get_top_k_neighbors(df, query_embedding, k=k_neighbors)

    # 3) build prompt
    prompt = build_rag_prompt_for_lyric_popularity(user_lyric, neighbors)

    # 4) call llm
    llm_output = call_llm_for_popularity_and_explanation(prompt)

    pred_pop = llm_output.get("predicted_popularity", None)
    explanation = llm_output.get("explanation", "")

    return {
        "predicted_popularity": pred_pop,
        "explanation": explanation,
        "neighbors_used": neighbors,
        "prompt_sent": prompt,
    }


# Now we can test the system

In [ ]:
import numpy as np
import pandas as pd

# 1. Grab the existing list (which currently has the bad columns)
# Note: Ensure this variable exists. If not, re-run the cell where it was defined originally.
# If you get a NameError here, look for the cell with 'audio_feature_cols = ...' and run it first.

# 2. Filter it to keep ONLY numeric columns
# We explicitly exclude 'lyrics', 'title', 'artist', etc. by checking the data type.
audio_feature_cols = [
    col for col in audio_feature_cols 
    if pd.api.types.is_numeric_dtype(df[col])
]

print(f"Sanitized feature list. Removed non-numeric columns.")
print(f"Count of valid audio features: {len(audio_feature_cols)}")

### I don't have an Open AI Key, so I built a mock LLM function. Remove to run with the Open AI cell!

In [ ]:
# --- MOCK LLM FUNCTION (No API Key Needed) ---
# This replaces the function from the skipped cell.
def call_llm_for_popularity_and_explanation(prompt: str) -> dict:
    print(" [Mock LLM] Prompt received! (Skipping OpenAI call)")
    
    # Return a fake response so the pipeline doesn't crash
    return {
        "predicted_popularity": 42, # Dummy score
        "explanation": (
            "SUCCESS: The pipeline ran successfully!\n"
            "Since you don't have an OpenAI API key, this is a placeholder explanation.\n"
            "However, the fact that you see this means your FAISS retrieval and "
            "data processing steps (Step 1 & 2) are working perfectly."
        )
    }

In [ ]:
test_lyric = "random test lyrics."
result = rag_lyric_popularity_system(df, test_lyric, k_neighbors=3)

print("Predicted popularity:", result["predicted_popularity"])
print("\nExplanation:\n", result["explanation"])


In [ ]:
# --- PRODUCTION CELL: GENERATE FULL INDEX ---
# NOTE TO TEAM: The HPC environment runs out of memory on the full 20k dataset.
# Please run this cell on a machine with 16GB+ RAM to generate the final files.

import faiss
import pickle

def generate_full_index():
    print("Generating full 20k index...")
    
    # 1. Prepare Data
    full_text = (
        df['title'].astype(str) + " " +
        df['artist'].astype(str) + " " +
        df['lyrics'].astype(str).str.slice(0, 500)
    ).tolist()
    
    # 2. Embed (Batch processing recommended if memory is tight)
    # Note: Using show_progress_bar=True is fine on local machines
    full_vectors = model.encode(full_text, show_progress_bar=True)
    
    # 3. Build Index
    dimension = 384
    final_index = faiss.IndexFlatL2(dimension)
    final_index.add(full_vectors.astype('float32'))
    
    # 4. Save
    faiss.write_index(final_index, "lyrics_20k.index")
    df.to_pickle("lyrics_20k_metadata.pkl")
    print("Files saved successfully.")

# Uncomment the line below to run generation:
# generate_full_index()